In [34]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from bs4 import BeautifulSoup as bs
import requests
import csv
import time
from datetime import datetime

The site I'm pulling data from:https://www.planecrashinfo.com/database.htm
For some reason it does not have a data table I can readily download, so I made a challenge for myself to extract the data from the site itself.

Through inspecting the source from https://www.planecrashinfo.com/1920/1920.htm, I was able to determine the data exists in html tables, meaning I can extract it with relative ease.

In [25]:
url = 'https://www.planecrashinfo.com/1920/1920.htm'
response = requests.get(url)
soup = bs(response.text, 'html.parser')


In [26]:
with open("preview.html", "w", encoding='utf-8') as file:
    file.write(soup.prettify())

Tried printing the output directly, but it seems vscode/jupyter can't display it properly. So I wrote the output to preview.html. It produced the exact table I wanted. Next step is to extract the data by filtering for html tables.

In [28]:
# find the tables in the site
table = soup.find('table')

# Open a new CSV file
with open('testOutput.csv', 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)

    # Write header row manually
    headers = ["Date", "Location/Operator", "Aircraft_Type/Registration", "Fatalities"]
    writer.writerow(headers)

    # Extract data from each row in the table and write to the CSV file
    for row in table.find_all('tr')[1:]:  # Skipping the header row
        columns = row.find_all('td')
        data = [col.text.strip() for col in columns]
        writer.writerow(data)  # Write data to CSV file

Upon closer inspection, the location / operator tab, as well as aircraft type / registration tab are not properly seperated, further modification need to be made to seperate them.
current table:

Date,Location/Operator,Aircraft_Type/Registration,Fatalities
08 Jan 1935,"Near Karachi, PakistanIndian Air Force",Wapiti?,0/2(15)
Expected result:

Date,Location,Operator,Aircraft_Type,Registration,Fatalities
08 Jan 1935,"Near Karachi, Pakistan","Indian Air Force",Wapiti?,0/2(15)

In [31]:
url = 'https://www.planecrashinfo.com/1920/1920.htm'
response = requests.get(url)
soup = bs(response.text, 'html.parser')

table = soup.find('table')

with open('testOutputCleaned.csv', 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)

    # Seperated headers
    headers = ["Date", "Location", "Operator", "Aircraft_Type", "Registration", "Fatalities"]
    writer.writerow(headers)

    for row in table.find_all('tr')[1:]:  # Skipping the header row
        columns = row.find_all('td')
        data = []

        # Date
        data.append(columns[0].get_text(strip=True))

        # Location and Operator
        loc_op = columns[1].get_text("|", strip=True).split("|")
        if len(loc_op) == 2:
            # If there are two elements after splitting, it's likely location and operator
            data.extend(loc_op)
        else:
            # Handle cases where the split doesn't work as expected
            data.extend([columns[1].get_text(strip=True), ""])

        # Aircraft Type and Registration
        type_reg = columns[2].get_text("|", strip=True).split("|")
        if len(type_reg) == 2:
            data.extend(type_reg)
        else:
            # Handle cases where the split doesn't work as expected
            data.extend([columns[2].get_text(strip=True), ""])

        # Fatalities
        data.append(columns[3].get_text(strip=True))

        writer.writerow(data)


Writing was successful. Now to adapt it for all the tables, from 1920 to 2023. A for loop should do.

In [33]:
with open('data.csv', 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)

    # Write header row
    headers = ["Year", "Date", "Location", "Operator", "Aircraft_Type", "Registration", "Fatalities"]
    writer.writerow(headers)

    for year in range(1920, 2024):
        time.sleep(1)  # Delay to prevent server overload
        url = f'https://www.planecrashinfo.com/{year}/{year}.htm'
        response = requests.get(url)
        soup = bs(response.text, 'html.parser')

        table = soup.find('table')

        for row in table.find_all('tr')[1:]:  # Skipping the header row
            columns = row.find_all('td')
            data = [year]  # Start with the year

            # Date
            data.append(columns[0].get_text(strip=True))

            # Location and Operator
            loc_op = columns[1].get_text("|", strip=True).split("|")
            if len(loc_op) == 2:
                data.extend(loc_op)
            else:
                data.extend([columns[1].get_text(strip=True), ""])

            # Aircraft Type and Registration
            type_reg = columns[2].get_text("|", strip=True).split("|")
            if len(type_reg) == 2:
                data.extend(type_reg)
            else:
                data.extend([columns[2].get_text(strip=True), ""])

            # Fatalities
            data.append(columns[3].get_text(strip=True))

            writer.writerow(data)


The result still produce some errors. In the original site, there are some incorrect formatting and duplicated data points, which result in the scraped data being incorrectly formatted. for example: 
1926,08 Mar 1926,"Staaken, Germany",Deutche Lufthansa,Junkers F-13,"D-290
D-290",1/1(0)
As such, additional cleaning need to be done to correct the data.
In addition, changed date-time into actual date-time format, got rid of the 'year' column.

In [35]:
def clean_text(text):
    """Remove duplicate words and replace newlines with spaces. Probably the hardest part to implement."""
    text = ' '.join(text.split())  # Removes newlines and extra spaces
    words = text.split()
    # Remove duplicates while preserving order
    seen = set()
    return ' '.join([x for x in words if not (x in seen or seen.add(x))])

def parse_date(date_str):
    """Convert date string to a standard date-time format."""
    try:
        return datetime.strptime(date_str, '%d %b %Y').strftime('%Y-%m-%d')
    except ValueError:
        return date_str  # Return the original string if parsing fails

with open('data.csv', 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)

    # Write header row
    headers = ["Date", "Location", "Operator", "Aircraft_Type", "Registration", "Fatalities"]
    writer.writerow(headers)

    for year in range(1920, 2024):
        time.sleep(1)  # Delay to prevent server overload
        url = f'https://www.planecrashinfo.com/{year}/{year}.htm' # parse urls in order
        response = requests.get(url)
        soup = bs(response.text, 'html.parser')

        table = soup.find('table')

        for row in table.find_all('tr')[1:]:  # Skipping the header row
            columns = row.find_all('td')
            data = []

            # Date (converted to standard format)
            data.append(parse_date(clean_text(columns[0].get_text(strip=True))))

            # Location and Operator
            loc_op = clean_text(columns[1].get_text("|", strip=True)).split("|")
            data.extend(loc_op if len(loc_op) == 2 else [columns[1].get_text(strip=True), ""])

            # Aircraft Type and Registration
            type_reg = clean_text(columns[2].get_text("|", strip=True)).split("|")
            data.extend(type_reg if len(type_reg) == 2 else [columns[2].get_text(strip=True), ""])

            # Fatalities
            data.append(clean_text(columns[3].get_text(strip=True)))

            writer.writerow(data)
